In [12]:
import os
import re
import anthropic
import gradio as gr
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle


In [2]:
# Load environment variables.
dotenv_path = './example.env'
load_dotenv(dotenv_path)

# Store the API key in a variable.
ClaudeAPI_Key = os.getenv("ClaudeAPI_Key")


In [3]:
# Additional imports for list parser.
from langchain.output_parsers import CommaSeparatedListOutputParser

In [4]:
# Initialize the model.
chat = ChatAnthropic(temperature=0, api_key="ClaudeAPI_Key", model_name="claude-3-opus-20240229")

# Initialize the output parser.
parser = CommaSeparatedListOutputParser()

# Get the output format instructions.
instructions = parser.get_format_instructions()
print(instructions)

print()

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`



In [15]:
# Load your pre-trained model and tokenizer here
model = tf.keras.models.load_model('SecondRun.keras')

# Load the pickled tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

client = anthropic.Anthropic(api_key=ClaudeAPI_Key)

# Our definitions:
def analyze_and_respond(review):
    # Tokenize and pad the input review
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=50)
    
    # Get prediction from your model
    lstm_prediction = model.predict(padded_sequence)[0][0]
    lstm_sentiment = "positive" if lstm_prediction > 0.5 else "negative"
    
    # Create the initial message for Claude
    initial_message = f'''You are an AI assistant tasked with analyzing and responding to Google reviews. You will be provided with a Google review. Your job is to determine the sentiment, and then craft an appropriate response based on whether the review is positive or negative.

Here is the Google review you need to analyze:
<google_review>
{review}
</google_review>

First, carefully read the review and determine if it is positive or negative. 

In your analysis, consider the following:
- The overall tone of the review
- Any specific praise or complaints mentioned
- The star rating, if provided

Write your sentiment analysis inside <sentiment_analysis> tags. Include your reasoning and whether you think the review is positive or negative.

After your analysis, proceed based on whether the review is positive or negative:

For a positive review:
1. Express gratitude for the positive feedback
2. Highlight a specific point from the review, if applicable
3. Keep the tone warm and appreciative

For a negative review:
1. Express regret for the unsatisfactory experience
2. Acknowledge the specific issues mentioned in the review
3. Craft a response that:
   a. Addresses the concerns raised
   b. Explains steps being taken to improve
   c. Invites the customer to give the business another chance
4. Keep the tone professional, empathetic, and solution-oriented
5. Keep the response to a character limit of 500 characters.

Write your final response inside <response> tags.

Remember, your goal is to maintain a positive relationship with the customer, address their concerns, and uphold the reputation of the business.'''

    messages = [
        {"role": "user", "content": [{"type": "text", "text": initial_message}]}
    ]

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
        messages=messages
    )
    
    # Extract the text content from the response
    claude_analysis = response.content[0].text if isinstance(response.content, list) else response.content

    # Ensure claude_analysis is a string
    if not isinstance(claude_analysis, str):
        claude_analysis = str(claude_analysis)

    # Extract Claude's sentiment from the analysis
    claude_sentiment = "positive" if "positive" in claude_analysis.lower() else "negative"

    # Compare LSTM and Claude sentiments
    agreement = "Agree" if lstm_sentiment == claude_sentiment else "Disagree"

    # Extract Claude's response
    response_match = re.search(r'<response>(.*?)</response>', claude_analysis, re.DOTALL)
    claude_response = response_match.group(1).strip() if response_match else "No response found."

    # Create the final output
    final_output = f"Our LSTM and Claude {agreement} that this is a {claude_sentiment} review.\n\n{claude_response}"

    return final_output

def process_review(review):
    final_response = analyze_and_respond(review)
    return final_response, gr.update(visible=True)

#Gradio interface 

with gr.Blocks(theme='freddyaboulton/dracula_revamped') as demo:
    gr.Markdown("""
    <div style="display: flex; align-items: center;">
        <img src="https://www.logostack.com/wp-content/uploads/designers/ArtFreedom/12-01-4-900x600.jpg" alt="Logo" style="width: 50px; margin-right: 10px;">
        <h1>Silver Lining LLC</h1>
    </div>
    """)
    
    
    
    review_input = gr.Textbox(label="Insert Google Review")
    analyze_button = gr.Button("Analyze Review")
    
    final_output = gr.Textbox(label="Analysis and Response")
    
    with gr.Group(visible=False) as approval_group:
        approve_button = gr.Button("Approve Response")
    
    analyze_button.click(
        process_review,
        inputs=[review_input],
        outputs=[final_output, approval_group]
    )

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://010fce607b95a0faa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
